<a href="https://colab.research.google.com/github/Paola1234/graphQLtoCSV/blob/master/SignUps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Import all relevant packages and access [token generator](http://token.aiesec.org.br/get_token.php?token=c0aa46e01d77fb212fe0195636fb515f8e43b530087399ec49f) to generate a token as ***access_token***

In [0]:
import json
import requests
import re
import pandas as pd
from pandas.io.json import json_normalize
import pandas.io.json as pd_json


!pip install gql -q

from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

In [0]:
  token_get_url = "http://token.aiesec.org.br/get_token.php?token=c0aa46e01d77fb212fe0195636fb515f8e43b530087399ec49f"
  access_token_raw = requests.get(token_get_url)
  access_token = access_token_raw.text.strip()
  print('Setup complete! \nYour access token is ' + '"'+ access_token +'"')

Setup complete! 
Your access token is "8b2029d4a49d211afa9e3075bc0134b701e465cbaf5e2fc239b8e974a7bd5f5b"


In [0]:
print("Select start date (YYYY-MM-DD)")

while True:
    query_start_date = input()
    if re.match('^[0-9]{4}-[0-9]{2}-[0-9]{2}$', query_start_date):
        print("Valid input recorded")
        break
    print("invalid input")

Select start date (YYYY-MM-DD)


 2019-06-01


Valid input recorded


In [0]:
print("Select end date (YYYY-MM-DD)")

while True:
    query_end_date = input()
    if re.match('^[0-9]{4}-[0-9]{2}-[0-9]{2}$', query_end_date):
        print("Valid input recorded")
        break
    print("invalid input")

Select end date (YYYY-MM-DD)


 2019-09-15


Valid input recorded


# ICX RE Data

In [0]:
input_query_icx = """
{  allOpportunityApplication(per_page: 4000,filters: {experience_end_date: {from:"%sT22:00:00Z", to:"%sT22:00:00Z"},opportunity_home_mc :1606}page: 1) {
    data {
      id
      status
      nps_grade
      host_lc_name
      date_realized
      experience_end_date
      opportunity{
        programme{
          short_name
        }
      }
      person{
        home_lc{name} 
     }
      standards{
        constant_name
       	standard_option{
          option         
        }
      }
      			}
    		}
  }
  """% (query_start_date, query_end_date)

_transport = RequestsHTTPTransport(
    url='https://gis-api.aiesec.org/graphql?access_token='+str(access_token),
    use_json=True,
)
client = Client(
    transport=_transport,
    fetch_schema_from_transport=True,
)
#Here you insert the query
query_icx = gql(input_query_icx)

query_icx_json=client.execute(query_icx)


In [0]:
standards_data_icx = json_normalize(
    query_icx_json['allOpportunityApplication']['data'],
    record_path=['standards'],
    meta=['id','status','nps_grade', 'host_lc_name','date_realized', 'experience_end_date','person','opportunity'])

In [0]:
standards_data_icx['person'] = (standards_data_icx.loc[:, 'person']
                          .apply(lambda x: x['home_lc']['name']))

standards_data_icx['opportunity'] = (standards_data_icx.loc[:, 'opportunity']
                               .apply(lambda x: x['programme']['short_name']))

In [0]:
standards_data_icx.columns = ['standard','standard_answer','id', 'status','nps','host_lc','realized_date', 'experience_end_date', 'home_lc', 'programmee']

In [0]:
standards_data_icx['realized_date'] = pd.to_datetime(standards_data_icx['realized_date'])
standards_data_icx['experience_end_date'] = pd.to_datetime(standards_data_icx['experience_end_date'])

# OGX RE Data

In [0]:
input_query_ogx = """
{  allOpportunityApplication(per_page: 4000,filters: {experience_end_date: {from:"%sT22:00:00Z", to:"%sT22:00:00Z"},person_home_mc :1606}page: 1) {
    data {
      id
      status
      nps_grade
      host_lc_name
      date_realized
      experience_end_date
      opportunity{
        programme{
          short_name
        }
      }
      person{
        home_lc{name} 
     }
      standards{
        constant_name
       	standard_option{
          option         
        }
      }
      			}
    		}
  }
  """% (query_start_date, query_end_date)

_transport = RequestsHTTPTransport(
    url='https://gis-api.aiesec.org/graphql?access_token='+str(access_token),
    use_json=True,
)
client = Client(
    transport=_transport,
    fetch_schema_from_transport=True,
)
#Here you insert the query
query_ogx = gql(input_query_ogx)

query_ogx_json=client.execute(query_ogx)


In [0]:
standards_data_ogx = json_normalize(
    query_ogx_json['allOpportunityApplication']['data'],
    record_path=['standards'],
    meta=['id','status','nps_grade', 'host_lc_name','date_realized', 'experience_end_date','person','opportunity'])

In [0]:
standards_data_ogx['person'] = (standards_data_ogx.loc[:, 'person']
                          .apply(lambda x: x['home_lc']['name']))

standards_data_ogx['opportunity'] = (standards_data_ogx.loc[:, 'opportunity']
                               .apply(lambda x: x['programme']['short_name']))

In [0]:
standards_data_ogx.columns = ['standard','standard_answer','id', 'status','nps','host_lc','realized_date', 'experience_end_date', 'home_lc', 'programmee']

In [0]:
standards_data_ogx['realized_date'] = pd.to_datetime(standards_data_ogx['realized_date'])
standards_data_ogx['experience_end_date'] = pd.to_datetime(standards_data_ogx['experience_end_date'])

# Processing and downloading

concatenate in per process dataframes

create process columns

In [0]:
standards_data_ogx["process"] = "OGX"
standards_data_icx["process"] = "ICX"

reorder columns

In [0]:
standards_data_icx = standards_data_icx[['id', 'programmee','process','home_lc','host_lc','status','realized_date','experience_end_date','nps','standard','standard_answer']]
standards_data_ogx = standards_data_ogx[['id', 'programmee','process','home_lc','host_lc','status','realized_date','experience_end_date','nps','standard','standard_answer']]

replaces GCDP for GV

In [0]:
standards_data_ogx = standards_data_ogx.replace("GCDP", "GV")
standards_data_icx = standards_data_icx.replace("GCDP", "GV")

standards_data_ogx = standards_data_ogx.replace("GIP", "GT")
standards_data_icx = standards_data_icx.replace("GIP", "GT")

unstacking the results for better readability

In [0]:
pivot_data_icx = standards_data_icx.set_index(['id', 'programmee','process','home_lc','host_lc','status','realized_date','experience_end_date','nps','standard'])['standard_answer'].unstack()
unstack_data_icx = pd.DataFrame(pivot_data_icx.to_records())

In [0]:
pivot_data_ogx = standards_data_ogx.set_index(['id', 'programmee','process','home_lc','host_lc','status','realized_date','experience_end_date','nps','standard'])['standard_answer'].unstack()
unstack_data_ogx = pd.DataFrame(pivot_data_ogx.to_records())

### Download

In [0]:
len(standards_data_ogx["id"].unique().tolist())

947

In [0]:
unstack_data_ogx.to_csv('standards_ogx_{}_{}.csv'.format(query_start_date, query_end_date),index=False)
unstack_data_icx.to_csv('standards_icx_{}_{}.csv'.format(query_start_date, query_end_date),index=False)

error: Not all EPs are being called in the extraction. some realizations are missing

COMPLE